In [3]:
import pandas as pd
import numpy as np

from prophet import Prophet

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Project_1/travel_data/travel_data.csv')

In [6]:
df.head(3)

,ALL_KWRD_RANK_CO,SRCHWRD_NM,LWPRT_CTGRY_NM,AREA_NM,MOBILE_SCCNT_VALUE,PC_SCCNT_VALUE,SCCNT_SM_VALUE,SCCNT_DE
0,1,부산여행,국내여행,부산광역시,2470,1629,4099,2019-07-09
1,2,제주여행,국내여행,제주특별자치도,1775,764,2539,2019-07-09
2,3,여수여행,국내여행,전라남도,1309,511,1820,2019-07-09


In [8]:
df['SCCNT_DE'] = pd.to_datetime(df['SCCNT_DE'])

In [9]:
df['SCCNT_DE']

0       2019-07-09
1       2019-07-09
2       2019-07-09
3       2019-07-09
4       2019-07-09
           ...    
14006   2023-05-29
14007   2023-05-29
14008   2023-05-29
14009   2023-05-29
14010   2023-05-29
Name: SCCNT_DE, Length: 14011, dtype: datetime64[ns]

In [101]:
korean_df = df.loc[df['LWPRT_CTGRY_NM'] == '국내여행', ]
oversea_df = df.loc[df['LWPRT_CTGRY_NM'] == '해외여행',]

In [124]:
korean_df.loc[korean_df['SCCNT_DE'] == '2019-07-09',]

,ALL_KWRD_RANK_CO,SRCHWRD_NM,LWPRT_CTGRY_NM,AREA_NM,MOBILE_SCCNT_VALUE,PC_SCCNT_VALUE,SCCNT_SM_VALUE,SCCNT_DE
0,1,부산여행,국내여행,부산광역시,2470,1629,4099,2019-07-09
1,2,제주여행,국내여행,제주특별자치도,1775,764,2539,2019-07-09
2,3,여수여행,국내여행,전라남도,1309,511,1820,2019-07-09
3,4,강릉여행,국내여행,강원도,1083,435,1518,2019-07-09
4,5,경주여행,국내여행,경상북도,1078,430,1508,2019-07-09
5,6,속초여행,국내여행,강원도,797,328,1125,2019-07-09
6,7,남해여행,국내여행,경상남도,630,301,931,2019-07-09
7,8,전주여행,국내여행,전라북도,507,254,761,2019-07-09
8,9,대구여행,국내여행,대구광역시,520,206,726,2019-07-09
9,10,군산여행,국내여행,전라북도,481,243,724,2019-07-09


In [105]:
sorted(korean_df['AREA_NM'].unique())

['강원도',
 '경기도',
 '경상남도',
 '경상북도',
 '대구광역시',
 '대전광역시',
 '부산광역시',
 '울산광역시',
 '인천광역시',
 '전라남도',
 '전라북도',
 '제주특별자치도']

In [67]:
korea_list = sorted(korean_df['AREA_NM'].unique())

In [85]:
korean_df = korean_df.loc[korean_df['AREA_NM'] == korea_list[0], ['SCCNT_SM_VALUE', 'SCCNT_DE']]
korean_df = korean_df.groupby('SCCNT_DE')['SCCNT_SM_VALUE'].sum()

In [99]:
len(korean_df)

538

In [ ]:
korean_df_test = pd.DataFrame(
    {
        'y' : korean_df.values,
        'ds' : korean_df.index
    },
    index = range(korean_df.shape[0]),
)
korean_df_test

In [ ]:
korea_m_list = []
korea_forecast_dic_list = {}
for i in range(korean_df['AREA_NM'].nunique()):
  korea_m_list.append(Prophet())
  korean_prophet = korean_df.loc[korean_df['AREA_NM'] == korea_list[i], ['SCCNT_SM_VALUE', 'SCCNT_DE']]
  korean_prophet = korean_prophet.groupby('SCCNT_DE')['SCCNT_SM_VALUE'].sum()
  if len(korean_prophet) > 10:
    korean_prophet = pd.DataFrame(
        {
            'y' : korean_prophet.values,
            'ds' : korean_prophet.index
        },
        index = range(korean_prophet.shape[0]),
    )
    
    korea_m_list[i].fit(korean_prophet)
    future = korea_m_list[i].make_future_dataframe(periods = 365*3, freq='D')
    korea_forecast_dic_list[korea_list[i]] = korea_m_list[i].predict(future)

In [126]:
korea_forecast_list[6].head(3)

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2019-07-09,1381.126656,1691.873747,2412.574135,1381.126656,1381.126656,680.607168,680.607168,680.607168,10.530013,10.530013,10.530013,670.077155,670.077155,670.077155,0.0,0.0,0.0,2061.733824
1,2019-07-10,1380.893512,1678.579297,2401.238795,1380.893512,1380.893512,679.821449,679.821449,679.821449,-12.259107,-12.259107,-12.259107,692.080556,692.080556,692.080556,0.0,0.0,0.0,2060.714960
2,2019-07-19,1378.795212,1951.457660,2677.342896,1378.795212,1378.795212,945.494588,945.494588,945.494588,-55.454862,-55.454862,-55.454862,1000.949450,1000.949450,1000.949450,0.0,0.0,0.0,2324.289801


In [ ]:
for region, korea in korea_forecast_list.items():
  print(region)
  print(korea.loc[korea['ds']== '2019-07-09', 'yhat'])

In [ ]:
['강원도',
 '경기도',
 '경상남도',
 '경상북도',
 '대구광역시',
 '대전광역시',
 '부산광역시',
 '울산광역시',
 '인천광역시',
 '전라남도',
 '전라북도',
 '제주특별자치도']

In [ ]:
korea_m_list[6].plot(korea_forecast_list[6], xlabel='Date', ylabel='Price')

In [58]:
sorted(oversea_df['AREA_NM'].unique())[27]

'일본'

In [ ]:
oversea_df.loc[oversea_df['AREA_NM'] == '체코',]

In [136]:
oversea_list = sorted(oversea_df['AREA_NM'].unique())

In [ ]:
oversea_list

In [ ]:
oversea_m_list = []
oversea_forecast_dic_list = {}
for i in range(oversea_df['AREA_NM'].nunique()):
  oversea_m_list.append(Prophet())
  oversea_prophet = oversea_df.loc[oversea_df['AREA_NM'] == oversea_list[i], ['SCCNT_SM_VALUE', 'SCCNT_DE']]
  oversea_prophet = oversea_prophet.groupby('SCCNT_DE')['SCCNT_SM_VALUE'].sum()
  if len(oversea_prophet) > 10:
    oversea_prophet = pd.DataFrame(
        {
            'y' : oversea_prophet.values,
            'ds' : oversea_prophet.index
        },
        index = range(oversea_prophet.shape[0]),
    )

    oversea_m_list[i].fit(oversea_prophet)
    future = oversea_m_list[i].make_future_dataframe(periods = 365*3, freq='D')
    oversea_forecast_dic_list[oversea_list[i]] = oversea_m_list[i].predict(future)

In [ ]:
for region, oversea in oversea_forecast_dic_list.items():
  print(region)
  print(oversea.loc[oversea['ds']== '2022-07-09', 'yhat'])

In [ ]:
korea_m_list[0].plot(korea_forecast_list[0], xlabel='Date', ylabel='Price')

In [ ]:
oversea_forecast_list[27]

In [151]:
pip list | grep joblib

joblib                        1.2.0


In [152]:
import joblib

In [154]:
joblib.dump({
    '국내여행 학습 데이터' : korea_m_list,
    '국내여행 예측' : korea_forecast_dic_list,
    '해외 여행 학습 데이터': oversea_m_list,
    '해외여행 예측' : oversea_forecast_dic_list
}, 'travel.pkl')

['travel.pkl']